In [4]:

import numpy as np
from abc import abstractmethod
from sklearn.base import BaseEstimator

class Classifier:

    @abstractmethod
    def fit(self,X,y):
        pass

    @abstractmethod
    def predict(self,X):
        pass

class ClassifEuclid(Classifier, BaseEstimator):
    
    def __init__(self,labels=[]):
        """Constructor de la clase
        labels: lista de etiquetas de esta clase (argumento necesario)"""
        self.labels = labels
        self.Z = None # Array de centroides

    def fit(self, X, y):
        """Entrena el clasificador
        X: matriz numpy cada fila es un dato, cada columna una medida
        y: vector de etiquetas, tantos elementos como filas en X
        retorna objeto clasificador"""
        n = np.zeros(len(self.labels)) # Contador de ocurrencias de cada clase
        self.Z = np.zeros((len(self.labels), X.shape[1]))
        # Calcular la media: 
        # Sumar las ocurrencias de cada clase en self.Z
        for yi, Xi in zip(y, X):
            n[yi] = n[yi] + 1
            self.Z[yi] = self.Z[yi] + Xi
        # Dividir cada sumatorio entre el númeo de ocurrencias
        self.Z = self.Z / n[:,np.newaxis]
        return self

    def predict(self, X):
        """Estima el grado de pertenencia de cada dato a todas las clases 
        X: matriz numpy cada fila es un dato, cada columna una medida del vector de caracteristicas. 
        Retorna una matriz, con tantas filas como datos y tantas columnas como clases tenga
        el problema, cada fila almacena los valores pertenencia de un dato a cada clase"""
        # Calcular la distancia de cada fila a cada centroide
        #return np.sqrt(np.sum(np.power(X[:,np.newaxis] - self.Z, 2), axis=2))
        aux = X[:,None]-self.Z
        return np.sqrt(np.einsum('abc,abc->ab', aux, aux))

    def pred_label(self, X):
        """Estima la etiqueta de cada dato. La etiqueta puede ser un entero o bien un string.
        X: matriz numpy cada fila es un dato, cada columna una medida
        retorna un vector con las etiquetas de cada dato"""
        # Devuelve un array con el índice con valor mínimo de cada fila.
        # Cada índice se corresponde con la clase a la que pertenece.
        return np.argmin(self.predict(X), axis=1)
    
    def score(self, X, y):
        return self.num_aciertos(X, y)/len(y)
    
    def num_aciertos(self, X, y):
        """Cuenta el numero de aciertos del clasificador para un conjunto de datos X.
        X: matriz de datos a clasificar
        y: vector de etiquetas correctas"""
        # Contar el número de datos iguales en ambos vectores
        return np.sum(self.pred_label(X)==y)

class ClassifEstadistico(Classifier, BaseEstimator):
    
    def __init__(self,labels=[]):
        """Constructor de la clase
        labels: lista de etiquetas de esta clase (argumento necesario)"""
        self.labels = labels
        self.mu = None # Array de medias
        self.cov = None # Array de matrices de covarianza de cada clase
        self.cov_inv = None # Array de matrices de covarianza inversas
        self.det = None # Array de determinantes de las matrices de covarianza
        self.a = None
        self.b = None
        self.c = None

    def fit(self,X,y):
        """Entrena el clasificador
        X: matriz numpy cada fila es un dato, cada columna una medida
        y: vector de etiquetas, tantos elementos como filas en X
        retorna objeto clasificador"""
        n_labels = len(self.labels)
        n_caracteristicas = X.shape[1]
        self.mu = np.empty((n_labels, n_caracteristicas))
        self.cov = np.empty((n_labels, n_caracteristicas, n_caracteristicas))
        self.cov_inv = np.empty((n_labels, n_caracteristicas, n_caracteristicas))
        self.det = np.empty(n_labels)
        self.a = np.empty((n_labels, n_caracteristicas, n_caracteristicas))
        self.b = np.empty((n_labels, n_caracteristicas))
        self.c = np.empty(n_labels)
        for c in range(len(self.labels)):
            self.cov[c] = np.cov(X[y==c], rowvar=False)
            self.mu[c] = np.mean(X[y==c], axis=0)
            self.cov_inv[c] = np.linalg.inv(self.cov[c])
            self.det[c] = np.linalg.det(self.cov[c])
            self.a[c] = -.5 * self.cov_inv[c]
            self.b[c] = self.mu[c].T @ self.cov_inv[c]
            self.c[c] = -.5 * (self.mu[c].T @ self.cov_inv[c] @ self.mu[c]) -.5 * np.log(self.det[c]) + np.log(np.sum(y==c)/X.shape[0])
        return self

    def predict(self,X):
        """Estima el grado de pertenencia de cada dato a todas las clases 
        X: matriz numpy cada fila es un dato, cada columna una medida del vector de caracteristicas. 
        Retorna una matriz, con tantas filas como datos y tantas columnas como clases tenga
        el problema, cada fila almacena los valores pertenencia de un dato a cada clase"""
        #return (np.diagonal(X@self.a@X.T, axis1=1, axis2=2) + self.b@X.T + self.c[:,np.newaxis]).T
        return np.einsum('ab,cdb,ad->ac', X, self.a, X) + np.einsum('ab,cb->ca', self.b, X) + self.c[None,:]

    def pred_label(self,X):
        """Estima la etiqueta de cada dato. La etiqueta puede ser un entero o bien un string.
        X: matriz numpy cada fila es un dato, cada columna una medida
        retorna un vector con las etiquetas de cada dato"""
        # Devuelve un array con el índice con valor mínimo de cada fila.
        # Cada índice se corresponde con la clase a la que pertenece.
        return np.argmax(self.predict(X), axis=1)

    def score(self, X, y):
        return self.num_aciertos(X, y)/len(y)
    
    def num_aciertos(self,X,y):
        """Cuenta el numero de aciertos del clasificador para un conjunto de datos X.
        X: matriz de datos a clasificar
        y: vector de etiquetas correctas"""
        # Contar el número de datos iguales en ambos vectores
        return np.sum(self.pred_label(X)==y)


# Entrenamiento, predicción y evaluación de iris, wine y cancer

In [5]:
from sklearn.datasets import load_iris, load_wine, load_breast_cancer
from sklearn.neighbors import NearestCentroid
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.model_selection import cross_val_score

"""
Clasificador distancia euclídea
"""
print("#"*76, "\tClasificador distancia euclídea", "#"*76, sep="\n")
dataset = load_iris()
X = dataset.data
y = dataset.target

# 1. Cargar los datos de la base de datos de entrenamiento
a = ClassifEuclid(dataset.target_names)

# 2. Entrenar el clasificador
train = a.fit(np.array(X), y)

# 3. Predecir y evaluar el clasificador calculando el porcentaje de datos correctamente clasificados
n_aciertos = a.num_aciertos(X, y)
print("\t", "-"*68, "\n\tiris:   Aciertos: ", n_aciertos, "/", len(y), " (", "%.2f" % ((n_aciertos / len(y))*100), "%)", sep='')

# Evaluación por resustitución y validación cruzada
print("\t\tEvaluación por resustitución:", "%.4f" % a.score(X,y))
scores = cross_val_score(a, X, y, cv=5)
print("\t\tEvaluación por validación cruzada: ", "%.4f" % np.mean(scores), ", std:", "%.4f" % np.std(scores), sep='')

# Comparación con el clasificador de sklearn
nc = NearestCentroid()
nc.fit(X, y)
print("\t\t", "Clasificador sklearn: ", np.sum(nc.predict(X)==y), " aciertos", sep='')


# wine
dataset = load_wine()
X = dataset.data
y = dataset.target
b = ClassifEuclid(dataset.target_names)
train = b.fit(np.array(X), y)
n_aciertos = b.num_aciertos(X, y)
print("\t", "-"*68, "\n\twine:   Aciertos: ", n_aciertos, "/", len(y), " (", "%.2f" % ((n_aciertos / len(y))*100), "%)", sep='')
print("\t\tEvaluación por resustitución:", "%.4f" % b.score(X,y))
scores = cross_val_score(b, X, y, cv=5)
print("\t\tEvaluación por validación cruzada: ", "%.4f" % np.mean(scores), ", std:", "%.4f" % np.std(scores), sep='')
nc = NearestCentroid()
nc.fit(X, y)
print("\t\t", "Clasificador sklearn: ", np.sum(nc.predict(X)==y), " aciertos", sep='')


# cancer
dataset = load_breast_cancer()
X = dataset.data
y = dataset.target
c = ClassifEuclid(dataset.target_names)
train = c.fit(np.array(X), y)
n_aciertos = c.num_aciertos(X, y)
print("\t", "-"*68, "\n\tcancer: Aciertos: ", n_aciertos, "/", len(y), " (", "%.2f" % ((n_aciertos / len(y))*100), "%)", sep='')
print("\t\tEvaluación por resustitución:", "%.4f" % c.score(X,y))
scores = cross_val_score(c, X, y, cv=5)
print("\t\tEvaluación por validación cruzada: ", "%.4f" % np.mean(scores), ", std:", "%.4f" % np.std(scores), sep='')
nc = NearestCentroid()
nc.fit(X, y)
print("\t\t", "Clasificador sklearn: ", np.sum(nc.predict(X)==y), " aciertos", sep='')


"""
Clasificador estadístico
"""
print("\n", "#"*76, "\tClasificador estadístico", "#"*76, sep="\n")
# iris
dataset = load_iris()
X = dataset.data
y = dataset.target
b = ClassifEstadistico(dataset.target_names)
train = b.fit(np.array(X), y)
n_aciertos = b.num_aciertos(X, y)
print("\t", "-"*68, "\n\tiris:   Aciertos: ", n_aciertos, "/", len(y), " (", "%.2f" % ((n_aciertos / len(y))*100), "%)", sep='')
print("\t\tEvaluación por resustitución:", "%.4f" % b.score(X,y))
scores = cross_val_score(b, X, y, cv=5)
print("\t\tEvaluación por validación cruzada: ", "%.4f" % np.mean(scores), ", std:", "%.4f" % np.std(scores), sep='')
c_est = QuadraticDiscriminantAnalysis()
c_est.fit(X, y)
print("\t\t", "Clasificador sklearn: ", np.sum(c_est.predict(X)==y), " aciertos", sep='')

# wine
dataset = load_wine()
X = dataset.data
y = dataset.target
b = ClassifEstadistico(dataset.target_names)
train = b.fit(np.array(X), y)
n_aciertos = b.num_aciertos(X, y)
print("\t", "-"*68, "\n\twine:   Aciertos: ", n_aciertos, "/", len(y), " (", "%.2f" % ((n_aciertos / len(y))*100), "%)", sep='')
print("\t\tEvaluación por resustitución:", "%.4f" % b.score(X,y))
scores = cross_val_score(b, X, y, cv=5)
print("\t\tEvaluación por validación cruzada: ", "%.4f" % np.mean(scores), ", std:", "%.4f" % np.std(scores), sep='')
c_est = QuadraticDiscriminantAnalysis()
c_est.fit(X, y)
print("\t\t", "Clasificador sklearn: ", np.sum(c_est.predict(X)==y), " aciertos", sep='')

# cancer
dataset = load_breast_cancer()
X = dataset.data
y = dataset.target
b = ClassifEstadistico(dataset.target_names)
train = b.fit(np.array(X), y)
n_aciertos = b.num_aciertos(X, y)
print("\t", "-"*68, "\n\tcancer: Aciertos: ", n_aciertos, "/", len(y), " (", "%.2f" % ((n_aciertos / len(y))*100), "%)", sep='')
print("\t\tEvaluación por resustitución:", "%.4f" % b.score(X,y))
scores = cross_val_score(b, X, y, cv=5)
print("\t\tEvaluación por validación cruzada: ", "%.4f" % np.mean(scores), ", std:", "%.4f" % np.std(scores), sep='')
c_est = QuadraticDiscriminantAnalysis()
c_est.fit(X, y)
print("\t\t", "Clasificador sklearn: ", np.sum(c_est.predict(X)==y), " aciertos", sep='')


############################################################################
	Clasificador distancia euclídea
############################################################################
	--------------------------------------------------------------------
	iris:   Aciertos: 139/150 (92.67%)
		Evaluación por resustitución: 0.9267
		Evaluación por validación cruzada: 0.9133, std:0.0499
		Clasificador sklearn: 139 aciertos
	--------------------------------------------------------------------
	wine:   Aciertos: 129/178 (72.47%)
		Evaluación por resustitución: 0.7247
		Evaluación por validación cruzada: 0.7187, std:0.0804
		Clasificador sklearn: 129 aciertos
	--------------------------------------------------------------------
	cancer: Aciertos: 507/569 (89.10%)
		Evaluación por resustitución: 0.8910
		Evaluación por validación cruzada: 0.8841, std:0.0840
		Clasificador sklearn: 507 aciertos


############################################################################
	Clasificador estadí

Resultados de los tres experimentos (clasificador distancia euclídea):

| Base de datos | Número de aciertos | Porcentaje de aciertos |
| --- | --- | --- |
| Iris   | 139| 92.67|
| Wine   | 129| 72.47|
| Cancer | 507| 89.10|

Resultados de los tres experimentos (clasificador estadístico):

| Base de datos | Número de aciertos | Porcentaje de aciertos |
| --- | --- | --- |
| Iris   | 147| 98.00|
| Wine   | 177| 99.44|
| Cancer | 554| 97.37|


# Entrenamiento y evaluación de Isolet

In [6]:
import pandas as pd
import numpy as np
import os.path
from sklearn.datasets import fetch_openml

# Si existe la base de datos, cargo las variables
if os.path.exists("isolet_X.pickle"):
    X = pd.read_pickle('isolet_X.pickle')
    y = pd.read_pickle('isolet_y.pickle')
else:
    # Cargamos desde internet ( https://www.openml.org ) y la guardamos en el directorio local
    X, y = fetch_openml('isolet', version=1, return_X_y=True, cache=False)
    # Guardamos los datos para no volver a descargarlos
    X.to_pickle("isolet_X.pickle")
    y.to_pickle("isolet_y.pickle")

X_train = np.array(X[:6238])
y_train = pd.factorize(y)[0][:6238]
X_test = np.array(X[6238:])
y_test = pd.factorize(y)[0][6238:]

X = np.array(X)
y = pd.factorize(y)[0]

# Clasificador distancia euclidea
clss_euc = ClassifEuclid(np.unique(y_train))
clss_euc.fit(X_train, y_train)
# Clasificador estadístico
clss_est = QuadraticDiscriminantAnalysis()
clss_est.fit(X_train, y_train)

print("Clasificador distancia euclídea:")
print("\tEvaluación por resustitución:", "%.4f" % clss_euc.score(X_train, y_train))
print("\tEvaluación por exclusión:", "%.4f" % clss_euc.score(X_test, y_test))
print("Clasificador estadístico:")
print("\tEvaluación por resustitución:", "%.4f" % clss_est.score(X_train, y_train))
print("\tEvaluación por exclusión:", "%.4f" % clss_est.score(X_test, y_test))


c:\users\jan\appdata\local\programs\python\python39\lib\site-packages\sklearn\discriminant_analysis.py:873: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


Clasificador distancia euclídea:
	Evaluación por resustitución: 0.8809
	Evaluación por exclusión: 0.8743
Clasificador estadístico:
	Evaluación por resustitución: 1.0000
	Evaluación por exclusión: 0.0648


# Resumen de resultados

Resultados de los experimentos con el clasificador de la distancia euclídea:

| Base de datos | Acierto resustitucion | Acierto validacion cruzada | Acierto exclusion |
| --- | --- | --- | --- |
| Iris   | 0.9267 | 0.9133 | - |
| Wine   | 0.7247 | 0.7187 | - |
| Cancer | 0.8910 | 0.8841 | - |
| Isolet |  0.8809 | - | 0.8743 |



Resultados de los experimentos con el clasificador estadístico bayesiano:

| Base de datos | Acierto resustitucion | Acierto validacion cruzada | Acierto exclusion |
| --- | --- | --- | --- |
| Iris   | 0.9800 | 0.9600 | - |
| Wine   | 0.9944 | 0.7108 | - |
| Cancer | 0.9736 | 0.9613 | - |
| Isolet |  1.0000 | - | 0.0648 |


## Comentarios sobre los resultados

Fijándonos en la columna de aciertos por sustitución, vemos que el estadístico comete menos errores en las 3 bases de datos,
siendo mas destacable la diferencia en cuanto al porcentaje de aciertos en las de Wine y Cancer, puesto que pasa de un 72,47%
a un 99,44% en Wine y de un 89,1% a un 97,36%. Sin embargo, este incremento de la tasa de aciertos no se corresponde con un
aumento real de la precisión del clasificador, si no con un error en la forma de evaluar el rendimiento de los clasificadores.
Al evaluar ambos por validación cruzada, comprobamos que el aumento no es tan grande como el obtenido evaluando solo por
resusitución, siendo incluso menor la precisión del estadístico en Wine a pesar de que era donde a priori se producia el mayor
aumento. A pesar del peor rendimiento obtenido por el bayesiano en Wine, no podemos obviar que ha desempeñado mejor su funcion
en las otras dos bases de datos, mostrando incluso un aumento de la tasas de aciertos desde un 88,41% a un 96,13% en Cancer.

En cuanto a los resultados obtenidos en Isolet, vemos que los resultados del euclideo concuerdan más con los resultados que se
esperan de un clasificador que los del bayesiano. Como podemos ver, en el bayesiano se ha obtenido un 100% de precision por
resustitución, sin embargo se ha obtenido un 6,48% por exclusion. Este fenómeno se debe a la existencia de variables linealmente
dependientes en el conjunto de entrenamiento de isolet. Debido a las variables colineares, es imposible hallar la inversa de la
matriz de covarianzas, por lo que estos datos no se tienen en cuenta afectando negativamente al rendimiento del bayesiano, sin
embargo esto no sucede con el euclideo puesto que no tiene en cuenta el factor de la dispersion de los datos